In [55]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from sklearn import preprocessing
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.wordnet import WordNetLemmatizer 
import json
import itertools
import datetime 
from sklearn.preprocessing import OneHotEncoder
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import json
import datetime

In [71]:
# path variables for KEVIN

path = "D:\Kevin Liang/Documents/1_UT_SENIOR/UT_AUSTIN_FALL_2017/EE_379K/Data_Science_Final_Project/Kickstarter_Data/"

In [57]:
def search_results(query):
    browser = webdriver.Firefox()
    browser.get(query)
    time.sleep(1)
    body = browser.find_element_by_tag_name('body')
    while True:
        elemsCount = browser.execute_script("return document.querySelectorAll('.stream-items > li.stream-item').length")

        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        try:
            WebDriverWait(browser, 20).until(
                lambda x: x.find_element_by_xpath(
                    "//*[contains(@class,'stream-items')]/li[contains(@class,'stream-item')]["+str(elemsCount+1)+"]"))
        except:
            break

    errors = browser.find_elements_by_class_name('center')
    for error in errors:
        if (error.text == "Sorry! This person is no longer active on Kickstarter."):
            browser.quit()
            return 0, 0

    price = browser.find_elements_by_class_name('count')
    check = False
    for val in price:
        count = val.text
        count =count.encode('utf-8')
        if (check == True):
            num_backed= count
            check = False
        else:
            num_created = count
            check = True
            
    '''  
    projects = browser.find_elements_by_class_name('backed')
    for project in projects:
        text = project.text
        words = text.split()
        num_backed = str(words[1])
        #print num_backed
        browser.quit()
        
    '''  
    
    return int(num_backed), int(num_created)

In [58]:
def add_user_profile_data(df):
    user_num_backers = []
    user_num_creators = []
    
    for user_url in df['user_profiles']:
        b, c = search_results(user_url)
        user_num_backers.append(b)
        user_num_creators.append(c)
    
    df['user_num_backers'] = pd.Series(user_num_backers)
    df['user_num_creators'] = pd.Series(user_num_creators)
    
    return df

In [67]:
# data - Data set
# feature - feature that is going to be processed using nlp

def nlp_data(data, feature, total_words):
    all_words = []
    regex_word = "[^a-zA-Z]"
    lem = WordNetLemmatizer()

    for cell_data in data[feature]:
        words = re.sub(regex_word," ",cell_data).lower().split()
        words = [i for i in words if not i in stopwords.words('english')]
        # might change len of word
        words = [i for i in words if len(i) > 1]
        words = [lem.lemmatize(i, "v") for i in words]
        words = [lem.lemmatize(i) for i in words]
        all_words += words

    vectorizer = CountVectorizer(analyzer = "word", stop_words = "english", max_features = total_words)
    word_count = vectorizer.fit_transform(all_words).toarray()
    final_words = vectorizer.get_feature_names()
    
    return final_words

In [69]:
def preprocess_nlp(data, feature):
    word_list = []
    for i in data[feature]:
        words = re.sub(regex_word," ",i).lower().split()
        words = [i for i in words if not i in stopwords.words('english')]
        # might change len of word
        words = [i for i in words if len(i) > 1]
        words = [lem.lemmatize(i, "v") for i in words]
        words = [lem.lemmatize(i) for i in words]
        word_list.append(words)
    data.drop(feature, axis = 1)
    data[feature] = word_list
    return data

In [76]:
def nlp(data, list_of_features, total_words):
    for i in list_of_features:
        final_words = nlp_data(data, i, total_words)
        data = preprocess_nlp(data,i)
        nlp_features = []
        for word in final_words:
            lst = []
            for x in data[i]:
                lst.append(x.count(word))
            nlp_features.append(lst)
            
        nlp_features = np.array(nlp_features)
        nlp_features_column = np.array(nlp_features).T
        nlp_feature_data = pd.DataFrame(data = nlp_features_column, columns = final_words)
        data = data.drop(i, axis = 1)
        data = pd.concat([data, nlp_feature_data], axis = 1)
    return data

In [60]:
# data - dataset as a DataFrame object
# list_of_features - features that need to be one hot encoded

def ohe_data(data, list_of_features):
    ohe = data[list_of_features]
    new_data = data.drop(list_of_features, axis = 1)
    ohe_features = pd.get_dummies(ohe)
    return pd.concat([new_data,ohe_features], axis = 1)

In [61]:
def json_creator_parser(data):
    new_feature = []
    for i in range(data["creator"].shape[0]):
        json_parsed_data = json.loads(data["creator"].iloc[i])
        new_feature.append(json_parsed_data['urls']['web']['user'])
        
    new_data = data.drop('creator', axis = 1)
    new_data["user_profiles"] = new_feature
    return new_data


In [62]:
def json_category_parser(data):
    new_feature_category = []
    new_feature_subcategory = []
    for i in range(data["category"].shape[0]):
        json_parsed_data = json.loads(data['category'].iloc[i])
        new_feature_category.append(json_parsed_data['slug'].split('/')[0])
        new_feature_subcategory.append(json_parsed_data['slug'].split('/')[1])
        
    new_data = data.drop('category', axis = 1)
    new_data["category"] = new_feature_category
    new_data["subcategory"] = new_feature_subcategory
    return new_data

In [81]:
data = pd.read_csv(path + "kickstarter.csv", header = 0)
data = data.drop(["photo", "urls","source_url","location"], axis = 1)
total_time = abs(data["launched_at"] - data["deadline"])
data["total_time"] = total_time
nlp_features = ["name", "blurb"]
ohe_features = ["state","disable_communication","country","currency", "currency_trailing_code","is_starrable","current_currency","usd_type"]

In [64]:
data

,name,blurb,goal,pledged,state,slug,disable_communication,country,currency,currency_symbol,...,usd_type,creator,category,profile,spotlight,friends,is_starred,is_backing,permissions,total_time
id,,,,,,,,,,,,,,,,,,,,,
1361433686,"My Google Search History, le livre : Tome 2","Vous aimez Google, notre ami de tous les jours...",5000.0,5000.37,successful,my-google-search-history-le-livre-tome-2,False,FR,EUR,€,...,domestic,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",True,NaN,NaN,NaN,NaN,2592000
1851713855,Reimagined Pokemon: Conceptual Fan Art by Sofi...,"An on going project, re-imagining Nintendo's c...",100.0,127.00,successful,realistic-pokemon-conceptual-fan-art-by-sofia-...,False,GB,GBP,£,...,domestic,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",True,NaN,NaN,NaN,NaN,5187600
675977754,State of Exceptional Webnation,State of Exceptional Webnation is the future o...,12000.0,12536.32,successful,state-of-exceptional-webnation,False,CA,CAD,$,...,domestic,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",True,NaN,NaN,NaN,NaN,2797757
158101895,Marshmallow Pikachu Sticker Sheets~,Help me turn my Cute Marshmallow Pikachus into...,300.0,346.00,successful,marshmallow-pikachu-sticker-sheets,False,US,USD,$,...,domestic,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",True,NaN,NaN,NaN,NaN,2595600
656509465,""" When Women were Birds"" Art Exhibition",An Art Project. An Exhibition. Raising awarene...,500.0,500.00,successful,when-women-were-birds-art-exhibition,False,GB,GBP,£,...,domestic,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":1,""should_show_fea...",True,NaN,NaN,NaN,NaN,2592000
2133083163,Motivational MSPaint Postcards,PLEASE NOTE: THE POSTCARDS HAVE NOW GONE TO TH...,75.0,179.00,successful,motivational-mspaint-postcards,False,GB,GBP,£,...,domestic,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",True,NaN,NaN,NaN,NaN,1209600
1854120130,3D EOD Bomb Suite Figure Files,The highly sought after 3D files for the Bomb ...,1000.0,1150.00,successful,3d-eod-bomb-suite-figure-files,False,US,USD,$,...,domestic,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",True,NaN,NaN,NaN,NaN,1474658
459280232,2017 Abstract Pattern Calendar,I am funding the cost of printing 15 limited e...,225.0,405.00,successful,2017-abstract-pattern-calendar,False,US,USD,$,...,domestic,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",True,NaN,NaN,NaN,NaN,345600
1700107954,DTrang Comics Vol. 1,"A book combining my weekly comics #1-30, with ...",500.0,671.00,successful,dtrang-comics-vol-1,False,US,USD,$,...,domestic,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",True,NaN,NaN,NaN,NaN,1296000


In [82]:
data = ohe_data(data, ohe_features)
data = nlp(data,nlp_features, 100)
data = json_category_parser(data)
data = json_creator_parser(data)
#data = add_user_profile_data(data)
data


,id,goal,pledged,slug,currency_symbol,deadline,state_changed_at,created_at,launched_at,staff_pick,...,video,wall,want,way,work,world,year,category,subcategory,user_profiles
0,1361433686,5000.0,5000.37,my-google-search-history-le-livre-tome-2,€,1477591896,1477591901,1473709036,1474999896,True,...,0,0,0,0,0,0,0,art,digital art,https://www.kickstarter.com/profile/1828229310
1,1851713855,100.0,127.00,realistic-pokemon-conceptual-fan-art-by-sofia-...,£,1481389611,1481389611,1476182994,1476202011,False,...,0,0,0,0,0,0,0,art,digital art,https://www.kickstarter.com/profile/1423579673
2,675977754,12000.0,12536.32,state-of-exceptional-webnation,$,1479693600,1479693601,1475771563,1476895843,False,...,0,0,0,0,0,0,0,art,digital art,https://www.kickstarter.com/profile/336112221
3,158101895,300.0,346.00,marshmallow-pikachu-sticker-sheets,$,1480762373,1480762373,1478159622,1478166773,False,...,0,0,0,0,0,0,0,art,digital art,https://www.kickstarter.com/profile/1978272168
4,656509465,500.0,500.00,when-women-were-birds-art-exhibition,£,1481543007,1481543007,1478803559,1478951007,False,...,0,0,0,0,0,0,0,art,digital art,https://www.kickstarter.com/profile/77249453
5,2133083163,75.0,179.00,motivational-mspaint-postcards,£,1481204993,1481204994,1479746838,1479995393,False,...,0,0,0,0,0,0,0,art,digital art,https://www.kickstarter.com/profile/719824630
6,1854120130,1000.0,1150.00,3d-eod-bomb-suite-figure-files,$,1481936400,1481936401,1480379098,1480461742,False,...,0,0,0,0,0,0,0,art,digital art,https://www.kickstarter.com/profile/604459465
7,459280232,225.0,405.00,2017-abstract-pattern-calendar,$,1481910260,1481910262,1481560715,1481564660,False,...,0,0,0,0,0,0,0,art,digital art,https://www.kickstarter.com/profile/nicoleholm...
8,1700107954,500.0,671.00,dtrang-comics-vol-1,$,1483322340,1483322342,1480819847,1482026340,False,...,0,0,0,0,0,0,0,art,digital art,https://www.kickstarter.com/profile/1206538187
9,1494416871,425.0,545.00,the-quickest-way-to-a-mans-heart-limited-editi...,$,1482868800,1482868803,1482031407,1482039935,False,...,0,0,0,1,0,0,0,art,digital art,https://www.kickstarter.com/profile/966561248
